In [1]:
from torch import optim
import torch
import torchvision.transforms as transforms
from torchvision import datasets
import matplotlib.pyplot as plt 
import numpy as np
import torch.nn.functional as F
from torch import nn
import torchvision

In [2]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

In [76]:
train_dataset = torchvision.datasets.CIFAR10(root='data/',
                                             train=True,
                                             transform=transforms.ToTensor(),
                                             download=True)

training_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=80,
                                           shuffle=True, )
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transforms.ToTensor())
validation_loader = torch.utils.data.DataLoader(test_dataset, batch_size=80,
                                          shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [27]:
def fin_acc(validation_loader, device):
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    data_iter = iter(validation_loader)
    images, labels = next(data_iter)
    images = images.to(device)
    net.eval()
    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    gt = np.array([classes[labels[j]] for j in range(len(labels))])
    pred = np.array([classes[predicted[j]] for j in range(len(labels))])
    print(f'Accuracy is {(gt == pred).sum() / len(gt)}')

In [7]:
import torch.nn.functional as F
from torch import nn

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 8 * hidden_dim)
        self.dropout_1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(8 * hidden_dim, 6 * hidden_dim)
        self.dropout_2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(6 * hidden_dim, 4*hidden_dim)
        self.dropout_3 = nn.Dropout(0.3)
        self.fc4 = nn.Linear(4 * hidden_dim, hidden_dim)
        self.dropout_4=nn.Dropout(0.3)
        self.fc5 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout_1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout_2(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout_3(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.dropout_4(x) 
        x = self.fc5(x)
        return x

    def predict(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout_1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout_2(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout_3(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.dropout_4(x) 
        x = self.fc5(x)
        x = F.softmax(x)
        return x


net = Net(3072, 120, 10).to(device)
net.train()

Net(
  (fc1): Linear(in_features=3072, out_features=960, bias=True)
  (dropout_1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=960, out_features=720, bias=True)
  (dropout_2): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=720, out_features=480, bias=True)
  (dropout_3): Dropout(p=0.3, inplace=False)
  (fc4): Linear(in_features=480, out_features=120, bias=True)
  (dropout_4): Dropout(p=0.3, inplace=False)
  (fc5): Linear(in_features=120, out_features=10, bias=True)
)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)
num_epochs = 42
loss_history=[] 
correct_history=[] 
val_loss_history=[] 
val_correct_history=[] 

for e in range(num_epochs): 
    loss=0.0 
    correct=0.0 
    val_loss=0.0 
    val_correct=0.0 
    for inputs, labels in training_loader: 
        inputs = inputs.to(device) 
        labels = labels.to(device) 
        outputs = net(inputs) 
        loss1=criterion(outputs,labels) 
        optimizer.zero_grad() 
        loss1.backward() 
        optimizer.step() 
        _,preds=torch.max(outputs,1) 
        loss+=loss1.item() 
        correct+=torch.sum(preds==labels.data) 
    else: 
        with torch.no_grad(): 
            for val_input,val_labels in validation_loader: 
                val_input=val_input.to(device) 
                val_labels=val_labels.to(device) 
                val_outputs=net(val_input) 
                val_loss1=criterion(val_outputs,val_labels)  
                _,val_preds=torch.max(val_outputs,1) 
                val_loss+=val_loss1.item() 
                val_correct+=torch.sum(val_preds==val_labels.data) 
        epoch_loss=loss/len(training_loader) 
        epoch_acc=correct.float()/len(training_loader) 
        loss_history.append(epoch_loss) 
        correct_history.append(epoch_acc) 
        val_epoch_loss=val_loss/len(validation_loader) 
        val_epoch_acc=val_correct.float()/len(validation_loader) 
        val_loss_history.append(val_epoch_loss) 
        val_correct_history.append(val_epoch_acc) 
        print(f'Epoch - {e}')
        print('training_loss:{:.2f}, Accuracy - {:.2f}'.format(epoch_loss, epoch_acc.item())) 
        print('validation_loss:{:.2f}, Accuracy - {:.2f}'.format(val_epoch_loss,val_epoch_acc.item())) 

Epoch - 0
training_loss:2.30, Accuracy - 9.28
validation_loss:2.30, Accuracy - 10.56
Epoch - 1
training_loss:2.29, Accuracy - 12.06
validation_loss:2.27, Accuracy - 11.98
Epoch - 2
training_loss:2.18, Accuracy - 13.74
validation_loss:2.10, Accuracy - 15.07
Epoch - 3
training_loss:2.08, Accuracy - 15.76
validation_loss:2.06, Accuracy - 16.83
Epoch - 4
training_loss:2.03, Accuracy - 18.36
validation_loss:2.00, Accuracy - 20.21
Epoch - 5
training_loss:1.96, Accuracy - 21.75
validation_loss:1.94, Accuracy - 22.54
Epoch - 6
training_loss:1.91, Accuracy - 23.45
validation_loss:1.89, Accuracy - 24.48
Epoch - 7
training_loss:1.88, Accuracy - 24.96
validation_loss:1.86, Accuracy - 25.24
Epoch - 8
training_loss:1.85, Accuracy - 25.93
validation_loss:1.82, Accuracy - 27.06
Epoch - 9
training_loss:1.82, Accuracy - 26.75
validation_loss:1.80, Accuracy - 27.54
Epoch - 10
training_loss:1.79, Accuracy - 27.77
validation_loss:1.76, Accuracy - 28.90
Epoch - 11
training_loss:1.76, Accuracy - 28.69
valida

In [26]:
fin_acc(validation_loader, device)

Accuracy is 0.5625


In [75]:
import torch.nn.functional as F
from torch import nn

class Net(nn.Module):
    def __init__(self, chanal, clases):
        super().__init__()
        self.chanal=chanal
        self.conv1 = nn.Conv2d(chanal, chanal*6, chanal, 1,  padding=2)
        self.conv2 = nn.Conv2d(chanal*6, chanal*6*2, chanal, 1,padding=1)
        self.conv3 = nn.Conv2d(chanal*6*2, chanal*6*2*2, chanal, 1, padding=1)
        self.fc4 = nn.Linear(4*4*chanal*6*2*2, 4*4*chanal*6*2 )
        self.dropout_1 = nn.Dropout(0.3)
        self.fc5 = nn.Linear(4*4*chanal*6*2, 4*4*chanal*6*2)
        self.dropout_2 = nn.Dropout(0.3)
        self.fc6 = nn.Linear(4*4*chanal*6*2, clases)

    def forward(self, x):
        x=F.relu(self.conv1(x)) 
        x=F.max_pool2d(x,2,2) 
        x=F.relu(self.conv2(x)) 
        x=F.max_pool2d(x,2,2) 
        x=F.relu(self.conv3(x)) 
        x=F.max_pool2d(x,2,2) 
        x=x.view(-1,4*4*self.chanal*6*2*2) 
        x=F.relu(self.fc4(x)) 
        x=self.dropout_1(x) 
        x=self.fc5(x)
        x=self.dropout_2(x)
        x=self.fc6(x)
        return x

    def predict(self, x):
        x=F.relu(self.conv1(x)) 
        x=F.max_pool2d(x,2,2) 
        x=F.relu(self.conv2(x)) 
        x=F.max_pool2d(x,2,2) 
        x=F.relu(self.conv3(x)) 
        x=F.max_pool2d(x,2,2) 
        x=x.view(-1,4*4*self.chanal*6*2*2) 
        x=F.relu(self.fc4(x)) 
        x=self.dropout_1(x) 
        x=self.fc5(x)
        x=self.dropout_2(x)
        x=self.fc6(x)
        x = F.softmax(x)
        return x


net = Net(3, 10).to(device)
net.train()

Net(
  (conv1): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(18, 36, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(36, 72, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc4): Linear(in_features=1152, out_features=576, bias=True)
  (dropout_1): Dropout(p=0.3, inplace=False)
  (fc5): Linear(in_features=576, out_features=576, bias=True)
  (dropout_2): Dropout(p=0.3, inplace=False)
  (fc6): Linear(in_features=576, out_features=10, bias=True)
)

In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)
num_epochs = 42
loss_history=[] 
correct_history=[] 
val_loss_history=[] 
val_correct_history=[] 

for e in range(num_epochs): 
    loss=0.0 
    correct=0.0 
    val_loss=0.0 
    val_correct=0.0 
    for inputs, labels in training_loader: 
        inputs = inputs.to(device) 
        labels = labels.to(device) 
        outputs = net(inputs) 
        loss1=criterion(outputs,labels) 
        optimizer.zero_grad() 
        loss1.backward() 
        optimizer.step() 
        _,preds=torch.max(outputs,1) 
        loss+=loss1.item() 
        correct+=torch.sum(preds==labels.data) 
    else: 
        with torch.no_grad(): 
            for val_input,val_labels in validation_loader: 
                val_input=val_input.to(device) 
                val_labels=val_labels.to(device) 
                val_outputs=net(val_input) 
                val_loss1=criterion(val_outputs,val_labels)  
                _,val_preds=torch.max(val_outputs,1) 
                val_loss+=val_loss1.item() 
                val_correct+=torch.sum(val_preds==val_labels.data) 
        epoch_loss=loss/len(training_loader) 
        epoch_acc=correct.float()/len(training_loader) 
        loss_history.append(epoch_loss) 
        correct_history.append(epoch_acc) 
        val_epoch_loss=val_loss/len(validation_loader) 
        val_epoch_acc=val_correct.float()/len(validation_loader) 
        val_loss_history.append(val_epoch_loss) 
        val_correct_history.append(val_epoch_acc) 
        print(f'Epoch - {e}')
        print('training_loss:{:.2f}, Accuracy - {:.2f}'.format(epoch_loss, epoch_acc.item())) 
        print('validation_loss:{:.2f}, Accuracy - {:.2f}'.format(val_epoch_loss,val_epoch_acc.item())) 

Epoch - 0
training_loss:2.30, Accuracy - 8.85
validation_loss:2.30, Accuracy - 9.74
Epoch - 1
training_loss:2.30, Accuracy - 10.02
validation_loss:2.29, Accuracy - 10.97
Epoch - 2
training_loss:2.27, Accuracy - 12.14
validation_loss:2.22, Accuracy - 16.14
Epoch - 3
training_loss:2.13, Accuracy - 18.57
validation_loss:2.03, Accuracy - 21.58
Epoch - 4
training_loss:2.00, Accuracy - 21.87
validation_loss:1.95, Accuracy - 23.83
Epoch - 5
training_loss:1.93, Accuracy - 23.97
validation_loss:1.86, Accuracy - 26.38
Epoch - 6
training_loss:1.84, Accuracy - 26.88
validation_loss:1.76, Accuracy - 29.54
Epoch - 7
training_loss:1.74, Accuracy - 29.96
validation_loss:1.65, Accuracy - 32.73
Epoch - 8
training_loss:1.65, Accuracy - 32.63
validation_loss:1.59, Accuracy - 34.38
Epoch - 9
training_loss:1.57, Accuracy - 34.58
validation_loss:1.53, Accuracy - 36.14
Epoch - 10
training_loss:1.51, Accuracy - 36.45
validation_loss:1.48, Accuracy - 37.34
Epoch - 11
training_loss:1.46, Accuracy - 38.01
validat

In [42]:
fin_acc(validation_loader, device)

Accuracy is 0.725


In [77]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adagrad(net.parameters(), lr=0.01)
num_epochs = 42
loss_history=[] 
correct_history=[] 
val_loss_history=[] 
val_correct_history=[] 

for e in range(num_epochs): 
    loss=0.0 
    correct=0.0 
    val_loss=0.0 
    val_correct=0.0 
    for inputs, labels in training_loader: 
        inputs = inputs.to(device) 
        labels = labels.to(device) 
        outputs = net(inputs) 
        loss1=criterion(outputs,labels) 
        optimizer.zero_grad() 
        loss1.backward() 
        optimizer.step() 
        _,preds=torch.max(outputs,1) 
        loss+=loss1.item() 
        correct+=torch.sum(preds==labels.data) 
    else: 
        with torch.no_grad(): 
            for val_input,val_labels in validation_loader: 
                val_input=val_input.to(device) 
                val_labels=val_labels.to(device) 
                val_outputs=net(val_input) 
                val_loss1=criterion(val_outputs,val_labels)  
                _,val_preds=torch.max(val_outputs,1) 
                val_loss+=val_loss1.item() 
                val_correct+=torch.sum(val_preds==val_labels.data) 
        epoch_loss=loss/len(training_loader) 
        epoch_acc=correct.float()/len(training_loader) 
        loss_history.append(epoch_loss) 
        correct_history.append(epoch_acc) 
        val_epoch_loss=val_loss/len(validation_loader) 
        val_epoch_acc=val_correct.float()/len(validation_loader) 
        val_loss_history.append(val_epoch_loss) 
        val_correct_history.append(val_epoch_acc) 
        print(f'Epoch - {e}')
        print('training_loss:{:.2f}, Accuracy - {:.2f}'.format(epoch_loss, epoch_acc.item())) 
        print('validation_loss:{:.2f}, Accuracy - {:.2f}'.format(val_epoch_loss,val_epoch_acc.item())) 

Epoch - 0
training_loss:1.70, Accuracy - 30.39
validation_loss:1.53, Accuracy - 35.14
Epoch - 1
training_loss:1.30, Accuracy - 42.20
validation_loss:1.20, Accuracy - 45.70
Epoch - 2
training_loss:1.16, Accuracy - 46.78
validation_loss:1.17, Accuracy - 46.73
Epoch - 3
training_loss:1.07, Accuracy - 49.41
validation_loss:1.06, Accuracy - 49.88
Epoch - 4
training_loss:1.01, Accuracy - 51.50
validation_loss:1.04, Accuracy - 50.42
Epoch - 5
training_loss:0.96, Accuracy - 53.07
validation_loss:0.99, Accuracy - 51.64
Epoch - 6
training_loss:0.91, Accuracy - 54.32
validation_loss:0.98, Accuracy - 51.99
Epoch - 7
training_loss:0.87, Accuracy - 55.43
validation_loss:0.96, Accuracy - 53.00
Epoch - 8
training_loss:0.84, Accuracy - 56.32
validation_loss:0.96, Accuracy - 52.89
Epoch - 9
training_loss:0.81, Accuracy - 57.09
validation_loss:0.93, Accuracy - 53.66
Epoch - 10
training_loss:0.78, Accuracy - 58.08
validation_loss:0.91, Accuracy - 54.43
Epoch - 11
training_loss:0.76, Accuracy - 58.88
valid

In [78]:
fin_acc(validation_loader, device)

Accuracy is 0.7375


При использовании cnn слоев обучение модели ускорилось при этом вычислительной сложности сети стала ниже чем в случае полносвязной сетью так как у сверток 
  1) весов намного меньше 
  2) более гибкая связь (не все входы в равной степени влиют на выходы).  
  
Использование Adgard из семейства адаптированых градиентных спусков так же позволило увеличить скорость обучения на начальных эпохах за счет меньшего обновления весов которые ранее часто обновлялись.  
Так же использование дропаут (позволяет создать своего рода усредненый ""ансабль"" сетей за счет того что не все нейроны в равной степени задействованны при каждой эпохе) позволило стабилизировать обучение и уменьшить переобучение.

Цель по метрике 5 из 8 0.625 достигнута. Сверточная сеть показала 0.73 (ее можно еще дообучать а в случае адгард стоит увеличить количество нейронов в дропаут слоях и возможно довать L2 регуляризацию) В случае сетью с прямым распостранение тоже стоит продолжить обучение.